## Importação das bibliotecas

In [3]:
import sys
sys.path.append('../')

import pandas as pd


In [5]:
from src.duplicated_data import count_duplicate

In [41]:
df_consumo_energia_eletrica = pd.read_csv('../data/raw/consumo_energia_eletrica.csv',sep=',')
df_estado_regiao = pd.read_csv('../data/raw/estado_regiao.csv',sep=';', encoding='latin1')


#df_consumo_energia_eletrica.head()
df_estado_regiao.head()


,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil


In [42]:
df_estado_regiao.head()

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil


In [43]:
count_duplicate(dataframe=df_consumo_energia_eletrica)

df_consumo_energia_eletrica = df_consumo_energia_eletrica.drop_duplicates()

count_duplicate(dataframe=df_consumo_energia_eletrica)

foram encontrados 1017 dados duplicados
foram encontrados 0 dados duplicados


In [39]:
df_consumo_energia_eletrica.describe()
#df_estado_regiao.describe()


,ano,mes,numero_consumidores,consumo
count,38880.000000,38880.000000,25920.000000,38880.000000
mean,2013.500000,6.500000,682303.377199,611732.006687
std,5.766355,3.452097,1839311.687366,1208066.849193
min,2004.000000,1.000000,52.000000,354.000000
25%,2008.750000,3.750000,18956.500000,85476.000000
50%,2013.500000,6.500000,108174.500000,221062.000000
75%,2018.250000,9.250000,442412.000000,596699.000000
max,2023.000000,12.000000,19201264.000000,12809742.000000


### Merging datasets

In [46]:
merged_datasets = pd.merge(df_consumo_energia_eletrica,
                           df_estado_regiao,
                            how='inner',
                            left_on='sigla_uf',
                            right_on='sigla')

dropedNaN = merged_datasets.dropna()

dropedNaN.to_csv('../data/processed/dados_tratados.csv', 
                 sep=';', 
                 encoding='latin1'
                 )

dropedNaN.head()


,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo,id_estado,sigla,estado,regiao,pais
648,2004,1,RN,Outros,40857.000000,69617,20,RN,Rio Grande do Norte,Nordeste,Brasil
649,2004,1,SP,Outros,311650.000000,937538,25,SP,São Paulo,Sudeste,Brasil
650,2004,1,MS,Outros,56881.000000,67601,12,MS,Mato Grosso do Sul,Centro-Oeste,Brasil
651,2004,1,SC,Outros,226165.000000,209380,24,SC,Santa Catarina,Sul,Brasil
652,2004,1,RJ,Outros,70634.000000,416128,19,RJ,Rio de Janeiro,Sudeste,Brasil


In [34]:
colunas_ausentes = merged_datasets.columns[merged_datasets.isnull().any()] 

print("Colunas com dados ausentes:", colunas_ausentes)
 

Colunas com dados ausentes: Index(['numero_consumidores'], dtype='object')


In [9]:
filter = (df_consumo_energia_eletrica['sigla_uf'] == 'MG') & (df_consumo_energia_eletrica['ano'] == 2022)

data = df_consumo_energia_eletrica[filter]

data

,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo
35011,2022,1,MG,Total,NaN,4942127
35033,2022,2,MG,Total,NaN,4680352
35070,2022,3,MG,Total,NaN,5339645
35095,2022,4,MG,Total,NaN,5258199
35103,2022,5,MG,Total,NaN,5268416
...,...,...,...,...,...,...
36816,2022,8,MG,Residencial,7960092.000000,1003476
36842,2022,9,MG,Residencial,7974812.000000,1073754
36863,2022,10,MG,Residencial,8005574.000000,1103940
36894,2022,11,MG,Residencial,8020513.000000,1065384
